In [ ]:
#!eval-notebook "../Initialization/InitSystemorphBaseToMemory"

# Set Contest

In [ ]:
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
Workspace.Initialize(x => x.FromSource(DataSource));
await UploadCashflowsAsync("../Files/900.TransactionalData/NominalCashflows_CH_2020_12.csv");

In [ ]:
var scenario = "IRU";
var year = 2020;
var month = 12;
var reportingNode = "CH";
var dataNode = "DT1.1";
var format = ImportFormats.Cashflow;
Func<IfrsVariable, bool> filter = x => x.AmountType == "NIC" && x.DataNode == dataNode;

In [ ]:
var targetPartition = (await DataSource.Query<PartitionByReportingNodeAndPeriod>().Where(p => p.ReportingNode == reportingNode &&
                                                                                                   p.Year == year &&
                                                                                                   p.Month == month &&
                                                                                                   p.Scenario == scenario).ToArrayAsync()).Single().Id;
var defaultPartition = (await DataSource.Query<PartitionByReportingNodeAndPeriod>().Where(p => p.ReportingNode == reportingNode &&
                                                                                                    p.Year == year &&
                                                                                                    p.Month == month &&
                                                                                                    p.Scenario == null).ToArrayAsync()).Single().Id;


# Import Best Estimate and Scenario Cashflows

In [ ]:
var cashflows_MUP = $"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,MUP
@@Cashflow
DataNode,AmountType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,CL,C,,-110,0,0,-110,0,0,-110,0,0,-110,0,0,0,-110,0,0,-110,0,0,-110,0,0,-110,0
DT1.1,NIC,CL,C,,27.5,27.5,27.5,27.5,27.5,27.5,27.5,27.5,27.5,27.5,27.5,27.5,27.5,27.5,27.5,27.5,27.5,27.5,27.5,27.5,27.5,27.5,27.5,27.5
DT1.1,CU,CL,C,,11,11,11,11,11,11,11,11,11,11,11,11,11,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,3.3
DT1.1,RA,CL,C,,2.75,2.75,2.75,2.75,2.75,2.75,2.75,2.75,2.75,2.75,2.75,2.75,2.75,2.75,2.75,2.75,2.75,2.75,2.75,2.75,2.75,2.75,2.75,2.75";

In [ ]:
var cashflowsMUP_BE = $"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,MUP
@@Cashflow
DataNode,AmountType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,CL,C,,-100,0,0,-100,0,0,-100,0,0,-100,0,0,0,-100,0,0,-100,0,0,-100,0,0,-100,0
DT1.1,NIC,CL,C,,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25
DT1.1,CU,CL,C,,10,10,10,10,10,10,10,10,10,10,10,10,10,5,5,5,5,5,5,5,5,5,5,3
DT1.1,RA,CL,C,,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5";

In [ ]:
await Workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(defaultPartition);  
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(defaultPartition);
var defaultVar = Workspace.Query<IfrsVariable>().Where(filter).ToArray();
defaultVar

In [ ]:
await UploadCashflowsAsync("../Files/900.TransactionalData/NominalCashflows_CH_2020_12_Scenario.csv");
var scenarioVar = Workspace.Query<IfrsVariable>().Where(filter).ToArray();
(scenarioVar.Except(defaultVar, IfrsVariableComparer.Instance()).ToArray()).Length.Should().NotBe(0)

In [ ]:
var log = await UploadCashflowsAsync("../Files/900.TransactionalData/NominalCashflows_CH_2020_12_Scenario_BE.csv");
var mirrorBE = Workspace.Query<IfrsVariable>().Where(filter);
(mirrorBE.Except(defaultVar, IfrsVariableComparer.Instance()).ToArray()).Length.Should().Be(0)

# Import Best Estimate and Scenario Actuals

In [ ]:
var actualsMUP = $"
@@Main
ReportingNode,Year,Month,Scenario,
CH,2020,12,MUP,
@@Actual
DataNode,AocType,ValueType,AccidentYear,Value
DT1.1,CF,NIC,,308";

In [ ]:
var actualsMUP_BE = $"
@@Main
ReportingNode,Year,Month,Scenario,
CH,2020,12,MUP,
@@Actual
DataNode,AocType,ValueType,AccidentYear,Value
DT1.1,CF,NIC,,280";

In [ ]:
await UploadActualAsync("../Files/900.TransactionalData/Actuals_CH_2020_12.csv");
var defaultVar = Workspace.Query<IfrsVariable>().Where(filter).ToArray();

In [ ]:
await UploadActualAsync("../Files/900.TransactionalData/Actuals_CH_2020_12_Scenario.csv")
var scenarioVar = Workspace.Query<IfrsVariable>().Where(filter).ToArray();
(scenarioVar.Except(defaultVar, IfrsVariableComparer.Instance()).ToArray()).Length.Should().NotBe(0)

In [ ]:
await UploadActualAsync("../Files/900.TransactionalData/Actuals_CH_2020_12_Scenario_BE.csv")
var mirrorBE = Workspace.Query<IfrsVariable>().Where(filter);
(mirrorBE.Except(defaultVar, IfrsVariableComparer.Instance()).ToArray()).Length.Should().Be(0)